In [2]:
from datasets import load_dataset

/opt/conda/envs/klue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
my_data = load_dataset("../dataset/train", data_files="train.csv")

Using custom data configuration train-564fe555afb0b6d8
Found cached dataset csv (/opt/ml/.cache/huggingface/datasets/csv/train-564fe555afb0b6d8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 398.96it/s]


In [4]:
my_data["train"][0]

{'id': 0,
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': "{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}",
 'object_entity': "{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",
 'label': 'no_relation',
 'source': 'wikipedia'}

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

In [6]:
my_data = my_data.flatten()
my_data["train"][0]

{'id': 0,
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': "{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}",
 'object_entity': "{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",
 'label': 'no_relation',
 'source': 'wikipedia'}

In [7]:
def preprocess_function(examples):
    return tokenizer([x for x in examples["sentence"]], truncation=True)

In [8]:
my_data = my_data.map(preprocess_function,
                                batched=True,
                                num_proc=4,
                                remove_columns=my_data["train"].column_names)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/csv/train-564fe555afb0b6d8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-79fa98174ff5314d.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/csv/train-564fe555afb0b6d8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-60665fe9fbc4f26a.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/csv/train-564fe555afb0b6d8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-2d612fe1f0ec9f53.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/csv/train-564fe555afb0b6d8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-599652eab1756134.arrow


In [9]:
lm_dataset = my_data.map(group_texts, batched=True, num_proc=4)

NameError: name 'group_texts' is not defined

In [10]:
from transformers import DataCollatorForLanguageModeling

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [11]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer
import torch
torch.cuda.empty_cache()
model = AutoModelForMaskedLM.from_pretrained("monologg/koelectra-base-v3-generator")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    num_train_epochs=40,
    weight_decay=0.01,
    save_total_limit=5,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_data["train"],
    eval_dataset=my_data["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
old_collator = trainer.data_collator
trainer.data_collator = lambda data: dict(old_collator(data))
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 32470
  Num Epochs = 40
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 162360
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,2.109600,1.913725
2,2.020700,1.814314
3,1.973100,1.779774
4,1.937500,1.764220
5,1.932200,1.735819
6,1.877800,1.718128
7,1.846800,1.692992
8,1.845600,1.678616
9,1.807100,1.664502
10,1.771900,1.649551


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

In [21]:
from transformers import pipeline
model = AutoModelForMaskedLM.from_pretrained("./results/checkpoint-53000")
fill_mask = pipeline(task="fill-mask", tokenizer=tokenizer, model=model)

In [28]:
fill_mask("소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 [MASK]했다.")

[{'sequence': '소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 확고히 했다.',
  'score': 0.1746564358472824,
  'token': 23300,
  'token_str': '확고히'},
 {'sequence': '소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 분명히 했다.',
  'score': 0.11067641526460648,
  'token': 8370,
  'token_str': '분명히'},
 {'sequence': '소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 모른다고 했다.',
  'score': 0.07074985653162003,
  'token': 18159,
  'token_str': '모른다고'},
 {'sequence': '소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 명확히 했다.',
  'score': 0.06588910520076752,
  'token': 13724,
  'token_str': '명확히'},
 {'sequence': '소련군에서 그의 이름도 중국식의 진지첸으로 조선인으로서의 정체성도 함께 했다.',
  'score': 0.051871009171009064,
  'token': 6261,
  'token_str': '함께'}]